In [2]:
!pwd

/Users/sumitmishra/Desktop/GitHub/Credit-Default-Risk-Prediction


In [4]:
import os
import pandas as pd
pd.options.display.max_columns = None  # Show all columns
pd.options.display.width = 1000        # Adjust width to fit more data

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

import warnings
warnings.simplefilter('ignore', UserWarning)

from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

spark = SparkSession.builder.appName("Home Credit Default Prediction").getOrCreate()

spark.conf.set("spark.sql.repl.eagerEval.enabled", True) 
spark.conf.set("spark.sql.debug.maxToStringFields", 1000)
spark.conf.set("spark.sql.repl.eagerEval.maxNumCols", 200) 
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

def print_message(df):
    print ("Rows :: {}  |  Columns :: {}".format(df.count(), len(df.columns)))
    return (None)

path = os.getcwd()

25/01/27 22:08:54 WARN Utils: Your hostname, Sumits-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 10.61.203.73 instead (on interface en0)
25/01/27 22:08:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/27 22:08:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<img src="https://storage.googleapis.com/kaggle-media/competitions/home-credit/home_credit.png" width="800"/>

In [6]:
# Training and testing data
sdf_train = spark.read.option('header', True)\
           .csv(path + "/Data/application_train.csv")

sdf_test = spark.read.option('header', True)\
           .csv(path + "/Data/application_test.csv")

# Bureau data with previous credit history
sdf_bureau = spark.read.option('header', True)\
           .csv(path + "/Data/bureau.csv")

sdf_bureau_balance = spark.read.option('header', True)\
           .csv(path + "/Data/bureau_balance.csv")

# Previous application of the customers
sdf_prev_app = spark.read.option('header', True)\
           .csv(path + "/Data/previous_application.csv")

# Previous Home credit loan/cash loans with Home Credit
sdf_posh_cash_balance = spark.read.option('header', True)\
           .csv(path + "/Data/POS_CASH_BALANCE.csv")

# Credit Card Balance of the customers with Home Credit
sdf_cc_balance = spark.read.option('header', True)\
           .csv(path + "/Data/credit_card_balance.csv")

# previous installments by the customers
sdf_prev_installments = spark.read.option('header', True)\
           .csv(path + "/Data/installments_payments.csv")

# Column description
sdf_desc = spark.read.option('header', True)\
            .csv(path + "/Data/HomeCredit_columns_description.csv")

def table_desc(name, df = sdf_desc):
    
    df.filter(F.col('Row') == name)\
        .select('Table', 'Row', 'Description')\
        .distinct()\
        .show(10, False)
    return (None)

In [10]:
sdf_train.show(10, False)

+----------+------+------------------+-----------+------------+---------------+------------+----------------+----------+-----------+---------------+---------------+--------------------+-----------------------------+--------------------+-----------------+--------------------------+----------+-------------+-----------------+---------------+-----------+----------+--------------+---------------+----------------+----------+----------+---------------+---------------+--------------------+---------------------------+--------------------------+-----------------------+--------------------------+--------------------------+---------------------------+----------------------+----------------------+-----------------------+----------------------+-------------------+-------------------+-------------------+--------------+----------------+---------------------------+------------------+--------------+-------------+-------------+-------------+-------------+------------+--------------------+--------------+-

In [18]:
print_message(sdf_train)

Rows :: 307511  |  Columns :: 122


In [20]:
print_message(sdf_test)

Rows :: 48744  |  Columns :: 121


In [21]:
print_message(sdf_bureau)

Rows :: 1716428  |  Columns :: 17


In [24]:
print_message(sdf_prev_app)

Rows :: 1670214  |  Columns :: 37


In [9]:
sdf_bureau.show(10, False)

+----------+------------+-------------+---------------+-----------+------------------+-------------------+-----------------+----------------------+------------------+--------------+-------------------+--------------------+----------------------+---------------+------------------+-----------+
|SK_ID_CURR|SK_ID_BUREAU|CREDIT_ACTIVE|CREDIT_CURRENCY|DAYS_CREDIT|CREDIT_DAY_OVERDUE|DAYS_CREDIT_ENDDATE|DAYS_ENDDATE_FACT|AMT_CREDIT_MAX_OVERDUE|CNT_CREDIT_PROLONG|AMT_CREDIT_SUM|AMT_CREDIT_SUM_DEBT|AMT_CREDIT_SUM_LIMIT|AMT_CREDIT_SUM_OVERDUE|CREDIT_TYPE    |DAYS_CREDIT_UPDATE|AMT_ANNUITY|
+----------+------------+-------------+---------------+-----------+------------------+-------------------+-----------------+----------------------+------------------+--------------+-------------------+--------------------+----------------------+---------------+------------------+-----------+
|215354    |5714462     |Closed       |currency 1     |-497       |0                 |-153.0             |-153.0         

In [8]:
sdf_bureau_balance.show(10, False)

+------------+--------------+------+
|SK_ID_BUREAU|MONTHS_BALANCE|STATUS|
+------------+--------------+------+
|5715448     |0             |C     |
|5715448     |-1            |C     |
|5715448     |-2            |C     |
|5715448     |-3            |C     |
|5715448     |-4            |C     |
|5715448     |-5            |C     |
|5715448     |-6            |C     |
|5715448     |-7            |C     |
|5715448     |-8            |C     |
|5715448     |-9            |0     |
+------------+--------------+------+
only showing top 10 rows



In [103]:
sdf_prev_app.show(10, False)

+----------+----------+------------------+-----------+---------------+----------+----------------+---------------+--------------------------+-----------------------+---------------------------+----------------------+-----------------+---------------------+------------------------+----------------------+--------------------+-------------+---------------------+------------------+---------------+----------------+-------------------+--------------+-----------------+-----------------------+----------------+--------------------+-----------+----------------+------------------------+------------------+--------------+-------------------------+-------------+----------------+-------------------------+
|SK_ID_PREV|SK_ID_CURR|NAME_CONTRACT_TYPE|AMT_ANNUITY|AMT_APPLICATION|AMT_CREDIT|AMT_DOWN_PAYMENT|AMT_GOODS_PRICE|WEEKDAY_APPR_PROCESS_START|HOUR_APPR_PROCESS_START|FLAG_LAST_APPL_PER_CONTRACT|NFLAG_LAST_APPL_IN_DAY|RATE_DOWN_PAYMENT|RATE_INTEREST_PRIMARY|RATE_INTEREST_PRIVILEGED|NAME_CASH_LOAN_PURP

### Customer Example

In [26]:
sdf_train.filter(F.col('SK_ID_CURR') == 380361)\
            .sort(F.asc('SK_ID_CURR'))\
            .show(20, False)

+----------+------+------------------+-----------+------------+---------------+------------+----------------+----------+-----------+---------------+---------------+----------------+-----------------------------+------------------+-----------------+--------------------------+----------+-------------+-----------------+---------------+-----------+----------+--------------+---------------+----------------+----------+----------+---------------+---------------+--------------------+---------------------------+--------------------------+-----------------------+--------------------------+--------------------------+---------------------------+----------------------+----------------------+-----------------------+-----------------+------------------+------------------+------------------+--------------+----------------+---------------------------+---------------+--------------+-------------+-------------+-------------+-------------+------------+--------------------+--------------+------------------

In [28]:
sdf_bureau.filter(F.col('SK_ID_CURR') == 380361)\
            .sort(F.asc('SK_ID_BUREAU'))\
            .show(20, False)

+----------+------------+-------------+---------------+-----------+------------------+-------------------+-----------------+----------------------+------------------+--------------+-------------------+--------------------+----------------------+---------------+------------------+-----------+
|SK_ID_CURR|SK_ID_BUREAU|CREDIT_ACTIVE|CREDIT_CURRENCY|DAYS_CREDIT|CREDIT_DAY_OVERDUE|DAYS_CREDIT_ENDDATE|DAYS_ENDDATE_FACT|AMT_CREDIT_MAX_OVERDUE|CNT_CREDIT_PROLONG|AMT_CREDIT_SUM|AMT_CREDIT_SUM_DEBT|AMT_CREDIT_SUM_LIMIT|AMT_CREDIT_SUM_OVERDUE|CREDIT_TYPE    |DAYS_CREDIT_UPDATE|AMT_ANNUITY|
+----------+------------+-------------+---------------+-----------+------------------+-------------------+-----------------+----------------------+------------------+--------------+-------------------+--------------------+----------------------+---------------+------------------+-----------+
|380361    |5180513     |Closed       |currency 1     |-1146      |0                 |743.0              |-539.0         

In [30]:
ls = sdf_bureau.filter(F.col('SK_ID_CURR') == 380361).select('SK_ID_BUREAU').rdd.flatMap(lambda x: x).collect()
print (ls)

['5715448', '5715449', '5715451', '5715452', '5715453', '5715454', '5715455', '5715456', '5715457', '5715458', '5715459', '5715460', '5715461', '5180513', '5715447', '5715450', '5715462']


In [33]:
sdf_bureau_balance.filter(F.col('SK_ID_BUREAU').isin(ls))\
            .sort(F.asc('SK_ID_BUREAU'))\
            .show(20, False)

+------------+--------------+------+
|SK_ID_BUREAU|MONTHS_BALANCE|STATUS|
+------------+--------------+------+
|5180513     |-7            |C     |
|5180513     |-8            |C     |
|5180513     |-9            |C     |
|5180513     |-10           |C     |
|5180513     |-11           |C     |
|5180513     |-12           |C     |
|5180513     |-13           |C     |
|5180513     |-14           |C     |
|5180513     |-15           |C     |
|5180513     |-16           |C     |
|5180513     |-17           |C     |
|5180513     |-18           |C     |
|5180513     |-19           |C     |
|5180513     |-20           |C     |
|5180513     |-21           |C     |
|5180513     |-22           |C     |
|5180513     |-23           |C     |
|5180513     |-24           |C     |
|5180513     |-25           |C     |
|5180513     |-26           |C     |
+------------+--------------+------+
only showing top 20 rows



In [37]:
sdf_prev_app.filter(F.col('SK_ID_CURR') == 380361)\
            .sort(F.col('AMT_CREDIT'))\
            .show(20, False)

+----------+----------+------------------+-----------+---------------+----------+----------------+---------------+--------------------------+-----------------------+---------------------------+----------------------+-------------------+---------------------+------------------------+----------------------+--------------------+-------------+---------------------+------------------+---------------+----------------+-------------------+--------------+-----------------+-----------------------+----------------+--------------------+-----------+----------------+------------------------------+------------------+--------------+-------------------------+-------------+----------------+-------------------------+
|SK_ID_PREV|SK_ID_CURR|NAME_CONTRACT_TYPE|AMT_ANNUITY|AMT_APPLICATION|AMT_CREDIT|AMT_DOWN_PAYMENT|AMT_GOODS_PRICE|WEEKDAY_APPR_PROCESS_START|HOUR_APPR_PROCESS_START|FLAG_LAST_APPL_PER_CONTRACT|NFLAG_LAST_APPL_IN_DAY|RATE_DOWN_PAYMENT  |RATE_INTEREST_PRIMARY|RATE_INTEREST_PRIVILEGED|NAME_CASH

In [41]:
sdf_posh_cash_balance.filter(F.col('SK_ID_CURR') == 380361)\
                    .sort(F.col('MONTHS_BALANCE'))\
                    .show(20, False)

+----------+----------+--------------+--------------+---------------------+--------------------+------+----------+
|SK_ID_PREV|SK_ID_CURR|MONTHS_BALANCE|CNT_INSTALMENT|CNT_INSTALMENT_FUTURE|NAME_CONTRACT_STATUS|SK_DPD|SK_DPD_DEF|
+----------+----------+--------------+--------------+---------------------+--------------------+------+----------+
|2837103   |380361    |-1            |24.0          |22.0                 |Active              |0     |0         |
|1910156   |380361    |-10           |24.0          |18.0                 |Active              |0     |0         |
|1910156   |380361    |-11           |24.0          |19.0                 |Active              |0     |0         |
|1910156   |380361    |-12           |24.0          |20.0                 |Active              |0     |0         |
|1910156   |380361    |-13           |24.0          |21.0                 |Active              |0     |0         |
|1910156   |380361    |-14           |24.0          |22.0                 |Activ

In [45]:
sdf_cc_balance.filter(F.col('SK_ID_CURR') == 380361)\
            .sort(F.col('MONTHS_BALANCE'))\
            .show(20, False)

+----------+----------+--------------+-----------+-----------------------+------------------------+--------------------+--------------------------+------------------------+-----------------------+-------------------+-------------------------+------------------------+-------------+--------------------+------------------------+--------------------+--------------------------+------------------------+-------------------------+--------------------+------+----------+
|SK_ID_PREV|SK_ID_CURR|MONTHS_BALANCE|AMT_BALANCE|AMT_CREDIT_LIMIT_ACTUAL|AMT_DRAWINGS_ATM_CURRENT|AMT_DRAWINGS_CURRENT|AMT_DRAWINGS_OTHER_CURRENT|AMT_DRAWINGS_POS_CURRENT|AMT_INST_MIN_REGULARITY|AMT_PAYMENT_CURRENT|AMT_PAYMENT_TOTAL_CURRENT|AMT_RECEIVABLE_PRINCIPAL|AMT_RECIVABLE|AMT_TOTAL_RECEIVABLE|CNT_DRAWINGS_ATM_CURRENT|CNT_DRAWINGS_CURRENT|CNT_DRAWINGS_OTHER_CURRENT|CNT_DRAWINGS_POS_CURRENT|CNT_INSTALMENT_MATURE_CUM|NAME_CONTRACT_STATUS|SK_DPD|SK_DPD_DEF|
+----------+----------+--------------+-----------+------------------

In [49]:
sdf_prev_installments.filter(F.col('SK_ID_CURR') == 380361)\
                    .sort(F.col('DAYS_ENTRY_PAYMENT'))\
                    .show(20, False)

+----------+----------+----------------------+---------------------+---------------+------------------+--------------+-----------+
|SK_ID_PREV|SK_ID_CURR|NUM_INSTALMENT_VERSION|NUM_INSTALMENT_NUMBER|DAYS_INSTALMENT|DAYS_ENTRY_PAYMENT|AMT_INSTALMENT|AMT_PAYMENT|
+----------+----------+----------------------+---------------------+---------------+------------------+--------------+-----------+
|2104755   |380361    |0.0                   |30                   |-125.0         |-125.0            |357.48        |357.48     |
|2104755   |380361    |0.0                   |29                   |-156.0         |-156.0            |357.48        |357.48     |
|2653129   |380361    |1.0                   |24                   |-1599.0        |-1624.0           |25430.04      |25430.04   |
|2653129   |380361    |1.0                   |23                   |-1629.0        |-1636.0           |26242.425     |26242.425  |
|2653129   |380361    |1.0                   |22                   |-1659.0        

## Preprocessing and finding out

In [16]:
sdf_train.select([F.count(F.when(F.isnan(c), c)).alias(c) for c in sdf_train.columns]).show()

+----------+------+------------------+-----------+------------+---------------+------------+----------------+----------+-----------+---------------+---------------+----------------+-------------------+------------------+-----------------+--------------------------+----------+-------------+-----------------+---------------+-----------+----------+--------------+---------------+----------------+----------+----------+---------------+---------------+--------------------+---------------------------+--------------------------+-----------------------+--------------------------+--------------------------+---------------------------+----------------------+----------------------+-----------------------+-----------------+------------+------------+------------+--------------+----------------+---------------------------+---------------+--------------+-------------+-------------+-------------+-------------+------------+--------------------+--------------+-----------------------+-----------------+----

In [20]:
sdf_train.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in sdf_train.columns]).show()

+----------+------+------------------+-----------+------------+---------------+------------+----------------+----------+-----------+---------------+---------------+----------------+-------------------+------------------+-----------------+--------------------------+----------+-------------+-----------------+---------------+-----------+----------+--------------+---------------+----------------+----------+----------+---------------+---------------+--------------------+---------------------------+--------------------------+-----------------------+--------------------------+--------------------------+---------------------------+----------------------+----------------------+-----------------------+-----------------+------------+------------+------------+--------------+----------------+---------------------------+---------------+--------------+-------------+-------------+-------------+-------------+------------+--------------------+--------------+-----------------------+-----------------+----

In [69]:
sdf_bureau.filter(F.col('SK_ID_CURR') == 215354).select('AMT_ANNUITY', 'SK_ID_CURR').show(20, False)

+-----------+----------+
|AMT_ANNUITY|SK_ID_CURR|
+-----------+----------+
|null       |215354    |
|null       |215354    |
|null       |215354    |
|null       |215354    |
|null       |215354    |
|null       |215354    |
|null       |215354    |
|null       |215354    |
|null       |215354    |
|null       |215354    |
|null       |215354    |
+-----------+----------+



In [67]:
sdf_train.filter(F.col('SK_ID_CURR') == 215354).select('AMT_ANNUITY', 'SK_ID_CURR').show(20, False)

+-----------+----------+
|AMT_ANNUITY|SK_ID_CURR|
+-----------+----------+
|21181.5    |215354    |
+-----------+----------+



In [43]:
sdf_bureau.select('CREDIT_CURRENCY').distinct().show(10, False)

+---------------+
|CREDIT_CURRENCY|
+---------------+
|currency 2     |
|currency 1     |
|currency 4     |
|currency 3     |
+---------------+



In [25]:
sdf_train.filter(F.col('SK_ID_CURR') == 215354).show(10, False)

+----------+------+------------------+-----------+------------+---------------+------------+----------------+----------+-----------+---------------+---------------+----------------+-------------------+------------------+-----------------+--------------------------+----------+-------------+-----------------+---------------+-----------+----------+--------------+---------------+----------------+----------+----------+---------------+---------------+--------------------+---------------------------+--------------------------+-----------------------+--------------------------+--------------------------+---------------------------+----------------------+----------------------+-----------------------+-----------------+------------------+-----------------+------------------+--------------+----------------+---------------------------+---------------+--------------+-------------+-------------+-------------+-------------+------------+--------------------+--------------+-----------------------+-----

In [59]:
for i in sdf_train.columns:
    for j in sdf_bureau.columns:
        if (i == j):
            print (i)

SK_ID_CURR
AMT_ANNUITY


In [55]:
type(sdf_train.columns)

list

In [13]:
sdf_train.count()

307511

In [11]:
sdf_bureau.count()

1716428

In [24]:
len(sdf_train.columns)

122